# Normalize names

In [1]:
import csv
import os
import re
from collections import defaultdict
from collections import Counter
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import networkx as nx
import pandas as pd
import requests
import seaborn as sns
from matplotlib import pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

# Autores

In [2]:
%%time

autores_df = pd.read_csv('autores2.csv', header=0)
autores_df = autores_df.fillna('')

CPU times: user 4.85 s, sys: 242 ms, total: 5.1 s
Wall time: 5.1 s


In [3]:
autores_df.shape

(3054360, 5)

In [4]:
autores_df.head()

,TITULO,PAIS_EJEMPLAR,AUTOR,PAIS_AUTOR,FECHA_PUB
0,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO,2010
1,Un bicho extraño,Spain,Mon Daporta,DESCONOCIDO,2010
2,Quisiera tener ...,Mexico,Giovanna Zoboli,DESCONOCIDO,2010
3,Johannes Gutenberg,Spain,Lluís Borràs Perelló,DESCONOCIDO,2010
4,La fantástica leyenda de: la princesa y el dragón,Spain,Sonia Alins,DESCONOCIDO,2010


## Normalize

In [5]:
pais_autores = autores_df.drop_duplicates(['AUTOR'])[['AUTOR', 'PAIS_AUTOR']]
pais_autores = pd.Series(list(pais_autores['PAIS_AUTOR']), index=pais_autores['AUTOR'])

In [6]:
pais_autores.head()

AUTOR
Mon Daporta             DESCONOCIDO
Giovanna Zoboli         DESCONOCIDO
Lluís Borràs Perelló    DESCONOCIDO
Sonia Alins             DESCONOCIDO
DESCONOCIDO             DESCONOCIDO
dtype: object

In [7]:
count_autores = autores_df['AUTOR'].value_counts()

In [8]:
count_autores.head()

DESCONOCIDO               409373
Gabriel García Márquez     14397
Colombia                   11526
Hergé                      11510
Anthony Browne              9653
Name: AUTOR, dtype: int64

In [9]:
def normalize(s):
    return s \
        .replace('á', 'a') \
        .replace('à', 'a') \
        .replace('é', 'e') \
        .replace('è', 'e') \
        .replace('í', 'i') \
        .replace('ì', 'i') \
        .replace('ó', 'o') \
        .replace('ò', 'o') \
        .replace('ú', 'u') \
        .replace('ù', 'u')

In [10]:
name_dict = {}

for name in pais_autores.index:
    norm = normalize(name)
    if norm != name and norm in pais_autores.index:
        pais_name = pais_autores[name]
        pais_norm = pais_autores[norm]
        if pais_name != 'DESCONOCIDO' and pais_norm == 'DESCONOCIDO':
            name_dict[norm] = name
        elif pais_name == 'DESCONOCIDO' and pais_norm != 'DESCONOCIDO':
            name_dict[name] = norm
        else:
            name_count = count_autores[name]
            norm_count = count_autores[norm]
            if name_count > norm_count:
                name_dict[norm] = name
            elif name_count < norm_count:
                name_dict[name] = norm

In [11]:
len(name_dict)

630

In [12]:
%%time

def process_autor(x):
    n = x['AUTOR']
    return name_dict.get(n, n)

autores_df['AUTOR'] = autores_df.apply(process_autor, axis=1)

CPU times: user 1min 22s, sys: 131 ms, total: 1min 22s
Wall time: 1min 22s


In [13]:
autores_df['AUTOR'].nunique()

144507

# Saving results

In [14]:
autores_df.to_csv('autores3.csv', index=False)